# Syllabus courses scraper

## Data needed to:
### `Course` model:
- name - name of the course
- field - field of the course 
- semester - `Semester` model
- ECTS - ects points of the course
- test_type - exam/test
- additional_info - information about course
- lecturer

### `Semester` model:
- field_by_year - `FieldByYear` model
- number - number of semester
- ECTS_required - required ects points

### `FieldByYear` model:
- field - `Field`
- start_date -
- end_date -

### `Field` model:
- name -
- faculty - `Faculty`
- formula -
- kind
- type - stacjonarne/nie
- description - description
- specialization - 
- G1_subject - 
- G2_subject - 

### `Faculty` model:
- name -
- building -

In [1]:
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver

import pandas as pd
import numpy as np
import time

In [ ]:
faculty_df = pd.DataFrame(columns=['name'])
field_df = pd.DataFrame(columns=['faculty_id', 'name', 'kind', 'type', 'description', 'specialization'])
field_by_year_df = pd.DataFrame(columns=['field_id', 'start_date'])
semester_df = pd.DataFrame(columns=['field_by_year_id', 'ects', 'number'])
course_df = pd.DataFrame(columns=['field_id', 'name', 'semester', 'ects', 'test_type', 'lecturer', 'description'])

base_url = "https://sylabusy.agh.edu.pl/"
driver = webdriver.Chrome()
driver.get(base_url)

fields_types = ['Stacjonarne', 'Niestacjonarne']
FIELDS_TYPES = [fields_types[0], fields_types[1]]
fields_level = ['Studia inżynierskie I stopnia', "Studia magisterskie inżynierskie II stopnia", "Studia licencjackie I stopnia", "Studia magisterskie II stopnia", "Studia podyplomowe"]
FIELDS_LEVEL = [fields_level[0], fields_level[2]]


def get_all_starting_years():
    rozpoczecie_ul = driver.find_element(By.XPATH, "//div[contains(., 'Rozpoczęcie studiów')]/following-sibling::ul[1]")
    link_elements = rozpoczecie_ul.find_elements(By.TAG_NAME, "a")
    hrefs = [el.get_attribute("href") for el in link_elements]
    return hrefs

def get_all_faculties_per_year():
    linki_wydzialow = []
    nazwy_wydzialow = []
    wszystkie_wydzialy = driver.find_element(By.XPATH, '/html/body/main/div/div/section/div[2]')
    linki = wszystkie_wydzialy.find_elements(By.CSS_SELECTOR, 'div.elements-department a')
    for link in linki:
        href = link.get_attribute("href")
        if link.text != "Semestr wyrównawczy":
            linki_wydzialow.append(href)
            nazwy_wydzialow.append(link.text)
    return linki_wydzialow, nazwy_wydzialow

def get_all_fields_per_faculty():
    main_content = driver.find_element(By.ID, "main-content")
    links_a = main_content.find_elements(By.CSS_SELECTOR, "li.student-view-content-list-item a")
    kierunki_hrefs = []
    for link in links_a:
        if link.get_attribute("innerText").split(',')[0] in FIELDS_LEVEL and link.get_attribute("innerText").split(',')[1][1:] in FIELDS_TYPES:
            kierunki_hrefs.append(link.get_attribute("href"))
    return kierunki_hrefs

def get_faculty_field_description():
    breadcrumb = driver.find_element(By.CSS_SELECTOR, "nav[aria-label='breadcrumb']")
    items = breadcrumb.find_elements(By.CSS_SELECTOR, "ol.breadcrumb li.breadcrumb-item")
    texts = [item.text.strip() for item in items[1:]]

    subtitle = driver.find_element(By.CSS_SELECTOR, ".section-subtitle")
    subtitle_list = [part.strip() for part in subtitle.text.split(",")]

    container = driver.find_element(By.ID, "nav-tab-info-panel")
    # first_paragraph = container.find_element(By.TAG_NAME, "p")
    # opis_kierunku = first_paragraph.get_attribute("innerText").strip()

    opis_kierunku = container.text.split('Opiekun')[0].strip().split('Zobacz pełny opis kierunku')[0].strip()

    texts.extend(subtitle_list)
    texts.append(opis_kierunku)
    return  texts

def get_semesters() -> tuple[webdriver.Chrome, webdriver.Chrome]:
    semestry = driver.find_element(By.ID, 'pills-tabContent') 
    semestry = semestry.find_elements(By.CSS_SELECTOR, ":scope > div")[1:]
    przyciski_semestry = driver.find_element(By.ID, 'syl-grid-tabs') 
    przyciski_semestry = przyciski_semestry.find_elements(By.TAG_NAME, "button")[1:]
    return semestry, przyciski_semestry

def get_specialization_name(specka: webdriver.Chrome, kind, field_name_):
    time.sleep(0.1)
    try:
        nazwa_specki = specka.find_element(By.TAG_NAME, "h2")
    except:
        specialization = ''
        if kind == fields_level[1] or kind == fields_level[3]:
            specialization = field_name_
    else:
        nazwa_specki = nazwa_specki.find_element(By.TAG_NAME, 'button')
        nazwa_specki = nazwa_specki.find_elements(By.TAG_NAME, "span")[1]
        specialization = nazwa_specki.get_attribute("textContent").strip()
    # print(kind, field_name_, specialization)
    return specialization

COURSES_BAD_PREFIXES = ('Język', 'Przedmiot humanistyczny', 'Przedmioty humanistyczne', 'Elective Humanistic',
                        'German', 'Russian', 'Spanish', 'French')

def get_valid_courses_buttons(specka:webdriver.Chrome):
    time.sleep(0.1)
    table = specka.find_element(By.TAG_NAME, 'table')
    tbodys = table.find_elements(By.TAG_NAME, 'tbody')
    valid_buttons = []

    for tbody in tbodys:
        # all_tr = tbody.find_elements(By.TAG_NAME, 'tr')   # tutaj moze pojawic sie odfiltrowanie przedmiotow np. humanistycznych, jezykow.
                                                            # trzeba przejsc przez wszystkie <tr> i sprawdzic czy maja atrybut 'class', jezeli nie to sa 
                                                            # to przedmioty do dodania (za wyjatkie 'Zasady wyboru', ale tam nie ma przycisku, to jest git).
                                                            # Jezeli maja np. 'class=syl-grid-group syl-group-199396-element', to dla drugiego elementu po split()
                                                            # szukamy przycisku z 'id=syl-group-199396', dla niego szukamy nazwy i sprawdzamy czy dodac.
        #TODO: ograniczyc jezyki
        all_buttons = tbody.find_elements(By.TAG_NAME, 'button')
        for course_button in all_buttons:
            if course_button.get_attribute('id') and course_button.get_attribute('aria-controls'):
                continue
            else:
                if course_button.get_attribute('textContent').strip().startswith(COURSES_BAD_PREFIXES):
                    continue
                else:
                    valid_buttons.append(course_button)
    return valid_buttons

def get_course_data(course_button:webdriver.Chrome):
    driver.execute_script("arguments[0].click();", course_button)
    time.sleep(0.5)
    try:
        modal = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "modal-content")))
        exit_button = modal.find_element(By.CLASS_NAME, "modal-header").find_element(By.TAG_NAME, "button")

        all_clean_descrpitions = []

        sylabus_content = modal.find_element(By.ID, "syllabus_content")
        divs = sylabus_content.find_elements(By.CSS_SELECTOR, ':scope > div')

        course_name = divs[0].text[:-23]
        if divs[0].text[-22:] == "Karta opisu przedmiotu":
            # course_name = divs[0].text[:-23]
            pass
        else:
            course_name = divs[0].text[:-26]
        time.sleep(0.2)

        lecturer = sylabus_content.find_element(By.CLASS_NAME, 'head-author-right')
        lecturer = lecturer.get_attribute('textContent').strip()

        table = sylabus_content.find_element(By.TAG_NAME, 'table')
        tds = table.find_elements(By.TAG_NAME, 'td')
        test_type_divs = tds[1].find_elements(By.TAG_NAME, 'div')
        test_type = test_type_divs[1].text
        ects = int(tds[2].text.split()[-1])

        tresci_programowe = sylabus_content.find_elements(By.CSS_SELECTOR, 'tbody.syllabus-data')[1]
        rows = tresci_programowe.find_elements(By.TAG_NAME, 'tr')
        for row in rows:
            tds = row.find_elements(By.CSS_SELECTOR, ':scope > td')
            td = tds[1]
            raw_text = td.get_attribute('innerText')
            clean_text = ' '.join(raw_text.split())
            all_clean_descrpitions.append(clean_text)
        description = ' '.join(all_clean_descrpitions)
        
        # print(description)
        driver.execute_script("arguments[0].click();", exit_button)
        return course_name, lecturer, test_type, ects, description
    except:
        return '', '', '', '', ''


starting_years_hrefs = [base_url]
starting_years_hrefs.extend(get_all_starting_years())

for href in starting_years_hrefs[:1]:       # LATA ROZPOCZECIA
    driver.get(href)
    time.sleep(0.1)
    faculties_hrefs, faculties_names = get_all_faculties_per_year()

    for faculty_href, faculty_name in zip(faculties_hrefs[:], faculties_names[:]):    # ILOSC WYDZIALOW
        if faculty_name not in faculty_df['name'].values:
            row = {'name': faculty_name}
            faculty_df = pd.concat([faculty_df, pd.DataFrame([row])], ignore_index=True)

        driver.get(faculty_href)
        time.sleep(0.1)
        fields_hrefs = get_all_fields_per_faculty()

        for field_href in fields_hrefs[:]:     # ILOSC KIERUNKOW
            driver.get(field_href)
            time.sleep(0.1)
            faculty_name, field_name, starting_year, kind, studies_type, description = get_faculty_field_description()
            semesters, semesters_buttons = get_semesters()

            for idx, (button, semester) in enumerate(zip(semesters_buttons, semesters)):    # ILOSC SEMESTROW
                semester_num = idx + 1
                driver.execute_script("arguments[0].click();", button)
                time.sleep(0.1)
                specializations = semester.find_elements(By.TAG_NAME, 'article')

                for specka in specializations:      # ILOSC SPECJALIZACJI
                    # print(field_name)
                    specialization_name = get_specialization_name(specka, kind, field_name)
                    time.sleep(0.1)
                    if field_name not in field_df['name'].values: # or specialization_name not in field_df['specialization'].values:
                        faculty_id = faculty_df[faculty_df['name'] == faculty_name].index.values[0]
                        row = {'faculty_id': faculty_id, 'name': field_name, 'kind': kind, 'type': studies_type, 'description': description, 'specialization': specialization_name}
                        field_df = pd.concat([field_df, pd.DataFrame([row])], ignore_index=True)
                    elif specialization_name not in field_df[field_df['name']==field_name]['specialization'].values:
                        faculty_id = faculty_df[faculty_df['name'] == faculty_name].index.values[0]
                        row = {'faculty_id': faculty_id, 'name': field_name, 'kind': kind, 'type': studies_type, 'description': description, 'specialization': specialization_name}
                        field_df = pd.concat([field_df, pd.DataFrame([row])], ignore_index=True)
                    elif studies_type not in field_df[field_df['name']==field_name]['type'].values:
                        faculty_id = faculty_df[faculty_df['name'] == faculty_name].index.values[0]
                        row = {'faculty_id': faculty_id, 'name': field_name, 'kind': kind, 'type': studies_type, 'description': description, 'specialization': specialization_name}
                        field_df = pd.concat([field_df, pd.DataFrame([row])], ignore_index=True)

                    courses_buttons = get_valid_courses_buttons(specka=specka)

                    for course_button in courses_buttons:
                        course_name, lecturer, test_type, ects, course_description = get_course_data(course_button)
                        print(course_name + '   ' + field_name)

                        field_id = field_df[(field_df['name'] == field_name) & (field_df['specialization'] == specialization_name) & (field_df['type'] == studies_type)].index.values[0]
                        row = {'field_id': field_id, 'name': course_name, 'lecturer': lecturer, 'semester': semester_num, 'test_type': test_type, 'ects': ects, 'description': course_description}
                        course_df = pd.concat([course_df, pd.DataFrame([row])], ignore_index=True)
                    
driver.quit()

Geologia   Budownictwo
Geometria wykreślna   Budownictwo
Grafika inżynierska i rysunek techniczny   Budownictwo
Geodezja   Budownictwo
Fizyka I   Budownictwo
Chemia (budowlana)   Budownictwo
Matematyka 1   Budownictwo
Technologie informacyjne   Budownictwo
Mechanika teoretyczna   Budownictwo
Ochrona własności intelektualnej   Budownictwo
Materiały budowlane   Budownictwo
Fizyka II   Budownictwo
Budownictwo ogólne   Budownictwo
Matematyka 2   Budownictwo
Eksploracja podwodna   Budownictwo
Historia i tradycje górnictwa   Budownictwo
Wszechświat, początek -ewolucja - człowiek   Budownictwo
Rysunek odręczny dla inżynierów   Budownictwo
Budownictwo ogólne   Budownictwo
Podstawy architektury i urbanistyki   Budownictwo
Fizyka budowli   Budownictwo
Mechanika gruntów   Budownictwo
Mechanika teoretyczna   Budownictwo
Technologia betonu   Budownictwo
Wytrzymałość materiałów   Budownictwo
Hydraulika i hydrologia   Budownictwo
Komputerowe wspomaganie projektowania   Budownictwo
Własności skał   Bu

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=135.0.7049.115)
Stacktrace:
	GetHandleVerifier [0x00007FF7898CEFA5+77893]
	GetHandleVerifier [0x00007FF7898CF000+77984]
	(No symbol) [0x00007FF7896991BA]
	(No symbol) [0x00007FF789671B63]
	(No symbol) [0x00007FF78971E92E]
	(No symbol) [0x00007FF78973E972]
	(No symbol) [0x00007FF789716F03]
	(No symbol) [0x00007FF7896E0328]
	(No symbol) [0x00007FF7896E1093]
	GetHandleVerifier [0x00007FF789B87B6D+2931725]
	GetHandleVerifier [0x00007FF789B82132+2908626]
	GetHandleVerifier [0x00007FF789BA00F3+3031443]
	GetHandleVerifier [0x00007FF7898E91EA+184970]
	GetHandleVerifier [0x00007FF7898F086F+215311]
	GetHandleVerifier [0x00007FF7898D6EC4+110436]
	GetHandleVerifier [0x00007FF7898D7072+110866]
	GetHandleVerifier [0x00007FF7898BD479+5401]
	BaseThreadInitThunk [0x00007FFA5440E8D7+23]
	RtlUserThreadStart [0x00007FFA55C3C5DC+44]


In [7]:
specialization_name

''

In [8]:
faculty_df

,name
0,Wydział Inżynierii Lądowej i Gospodarki Zasobami


In [9]:
field_df

,faculty_id,name,kind,type,description,specialization
0,0,Budownictwo,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Budownictwo należy do obszaru...,
1,0,Budownictwo,Studia inżynierskie I stopnia,Niestacjonarne,Kierunek studiów Budownictwo należy do obszaru...,


In [10]:
course_df

,field_id,name,semester,ects,test_type,lecturer,description
0,0,Geologia,1,3,Zaliczenie,Katarzyna Cyran,Zajęcia prowadzone w trybie mieszanym. Deforma...
1,0,Geometria wykreślna,1,2,Zaliczenie,Krzysztof Pałac,Ćwiczenia polegają na samodzielnym wykonywaniu...
2,0,Grafika inżynierska i rysunek techniczny,1,3,Zaliczenie,Sebastian Olesiak,Rysunek techniczny z zastosowaniem tradycyjnyc...
3,0,Geodezja,1,3,Zaliczenie,Michał Strach,1. Podstawowe pojęcia geodezyjne. Organizacja ...
4,0,Fizyka I,1,3,Zaliczenie,Michał Ślęzak,Wykład: Głównym celem wykładów jest jakościowe...
...,...,...,...,...,...,...,...
107,0,Mechanika gruntów,3,5,Egzamin,Michał Kowalski,Rozwiązywanie zadań dotyczących: * własności f...
108,0,Materiały budowlane,3,5,Egzamin,"Paweł Pichniarczyk, Agnies...",Podział materiałów budowlanych. Podstawowe wła...
109,0,Budownictwo ogólne,3,5,Egzamin,Sebastian Olesiak,Wykonanie uproszczonego projektu budowlanego b...
110,0,Metody obliczeniowe,4,2,Zaliczenie,Dorota Pawluś,Pojęcia podstawowe. Istota metod obliczeniowyc...


In [24]:
import re

course_df["description"] = (
    course_df["description"]
    .str.replace(r"[\r\n]+", " ", regex=True)   # usuń \n i \r
    .str.strip()
)

In [25]:
# faculty_df[faculty_df['nazwa'] == 'Wydział Odlewnictwa'].index.values[0]
# field_df.head()
# faculty_df.head()
course_df.tail()

,field_id,name,semester,ects,test_type,lecturer,description
823,11,Negocjacje w biznesie,1,2,Zaliczenie,Sławomir Ziółkowski,1.Istota negocjacji Negocjacje stanowiskowe a ...
824,11,Nowoczesne koncepcje zarządzania,1,6,Egzamin,Paweł Filipowicz,"1. Organizacja w różnych znaczeniach, ewolucja..."
825,11,Rachunkowość zarządcza,1,3,Zaliczenie,Marta Kołodziej-Hajdo,Koszty i przychody. Wynik finansowy. Rachunek ...
826,11,Społeczna odpowiedzialność biznesu,1,2,Zaliczenie,Joanna Kulczycka,1. Wpływ CSR na orgnizacje – praktyczne ćwicze...
827,11,Zarządzanie strategiczne,1,4,Egzamin,Alina Kozarkiewicz,Istota strategii organizacji i zarządzania str...


In [ ]:
import csv

faculty_df.to_csv('sylabus_data/faculty_data_2025_1_stopien.csv', sep=";", encoding="utf-8-sig", quoting=csv.QUOTE_ALL)
field_df.to_csv('sylabus_data/field_data_2025_1_stopien.csv', sep=";", encoding="utf-8-sig", quoting=csv.QUOTE_ALL)
course_df.to_csv('sylabus_data/course_data_2025_1_stopien.csv', sep=";", encoding="utf-8-sig", quoting=csv.QUOTE_ALL)

## Dataframe to db

In [24]:
faculty_df = pd.read_csv('faculty_data.csv', sep=";", encoding="utf-8-sig", quoting=csv.QUOTE_ALL)
field_df = pd.read_csv('field_data.csv', sep=";", encoding="utf-8-sig", quoting=csv.QUOTE_ALL)
course_df = pd.read_csv('course_data.csv', sep=";", encoding="utf-8-sig", quoting=csv.QUOTE_ALL)

In [14]:
import os
import django
import pandas as pd
import datetime
from django.db import transaction
from django.utils import timezone

In [ ]:
# ──────────────────────────────────────────────────────────────
# 1. Konfiguracja Django
# ──────────────────────────────────────────────────────────────
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend.settings")  # <- dostosuj w razie potrzeby
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"   #  ←  DODAJ TO
django.setup()

from django.core.management import call_command
call_command("makemigrations", "mainApp")
call_command("migrate")

from mainApp.models import (
    Building, Faculty, Field,
    FieldByYear, Semester, Course
)

# ──────────────────────────────────────────────────────────────
# 2. Zakładamy, że pięć DataFrame’ów już istnieje w namespace:
#    faculty_df, field_df, field_by_year_df, semester_df, course_df
# ──────────────────────────────────────────────────────────────
# Jeżeli chcesz testować standalone, odkomentuj i zbuduj mini-próbkę:
# faculty_df = pd.DataFrame([{"name": "Wydział Informatyki"}])
# field_df   = pd.DataFrame([{
#     "faculty_id": 0, "name": "Informatyka", "formula": "2*M+3*G1+G2",
#     "type": "STACJONARNE", "description": "", "specialization": ""
# }])
# field_by_year_df = pd.DataFrame([{"field_id": 0, "start_date": "2023-10-01"}])
# semester_df = pd.DataFrame([{"field_by_year_id": 0, "ects": 30, "number": 1}])
# course_df  = pd.DataFrame([{
#     "field_id": 0, "name": "Programowanie 1", "semester": 1,
#     "ects": 6, "test_type": "egzamin", "lecturer": "dr inż. Kowalski",
#     "description": "Podstawy Python"
# }])

# ──────────────────────────────────────────────────────────────
# 3. Domyślny budynek (utwórz, jeśli baza jest pusta)
# ──────────────────────────────────────────────────────────────
building, _ = Building.objects.get_or_create(
    id=1,
    defaults={"name": "Budynek domyślny"}
)
print(f"[+] Używam budynku: {building}")

# ──────────────────────────────────────────────────────────────
# 4. Import – wszystko w jednej transakcji
# ──────────────────────────────────────────────────────────────
with transaction.atomic():

    # 4a. FACULTY  ────────────────────────────────────────────
    faculty_map: dict[int, Faculty] = {}
    for idx, row in faculty_df.iterrows():
        name = str(row["name"]).strip()

        fac, created = Faculty.objects.get_or_create(
            name=name,
            defaults={"building": building},
        )
        if not created and fac.building_id != building.id:
            fac.building = building
            fac.save(update_fields=["building"])

        faculty_map[idx] = fac
        print(f"  {'[+] nowy' if created else '[=] istnieje'} Wydział {idx}: {name}")

    # 4b. FIELD (kierunek)  ───────────────────────────────────
    field_map: dict[int, Field] = {}
    for idx, row in field_df.iterrows():
        fac_idx = int(row["faculty_id"])
        fac_obj = faculty_map.get(fac_idx)
        if fac_obj is None:
            raise ValueError(f"Brakuje wydziału #{fac_idx} dla field_df[{idx}]")

        fld, created = Field.objects.get_or_create(
            name=str(row["name"]).strip(),
            faculty=fac_obj,
            defaults={
                "type":           row.get("type", "STACJONARNE") or "STACJONARNE",
                "description":    row.get("description", "") or "",
                "specialization": row.get("specialization", "") or "",
            },
        )
        # ewentualna aktualizacja pól „miękkich”
        if not created:
            dirty = False
            for attr in ("kind", "type", "description", "specialization"):
                new_val = row.get(attr, "") or ""
                if getattr(fld, attr) != new_val:
                    setattr(fld, attr, new_val); dirty = True
            if dirty:
                fld.save()

        field_map[idx] = fld
        print(f"  {'[+] nowy' if created else '[=] istnieje'} Kierunek {idx}: {fld.name}")

    # # 4c. FIELD-BY-YEAR  ──────────────────────────────────────
    # fby_map: dict[int, FieldByYear] = {}
    # for idx, row in field_by_year_df.iterrows():
    #     fld_idx = int(row["field_id"])
    #     fld_obj = field_map.get(fld_idx)
    #     if fld_obj is None:
    #         raise ValueError(f"Brakuje kierunku #{fld_idx} dla field_by_year_df[{idx}]")

    #     start_date = pd.to_datetime(row["start_date"]).date()
    #     # jeśli masz kolumnę 'end_date', użyj jej; w przeciwnym razie domyślnie 30 VI następnego roku
    #     end_date = (start_date.replace(year=start_date.year + 1, month=6, day=30)
    #                 if "end_date" not in row or pd.isna(row["end_date"])
    #                 else pd.to_datetime(row["end_date"]).date())

    #     fby, created = FieldByYear.objects.get_or_create(
    #         field=fld_obj,
    #         start_date=start_date,
    #         defaults={"end_date": end_date},
    #     )
    #     if not created and fby.end_date != end_date:
    #         fby.end_date = end_date
    #         fby.save(update_fields=["end_date"])

    #     fby_map[idx] = fby
    #     print(f"  {'[+] nowy' if created else '[=] istnieje'} FieldByYear {idx}: {fld_obj.name} {start_date.year}/{end_date.year}")

    # # 4d. SEMESTER  ───────────────────────────────────────────
    # sem_map: dict[int, Semester] = {}
    # for idx, row in semester_df.iterrows():
    #     fby_idx = int(row["field_by_year_id"])
    #     fby_obj = fby_map.get(fby_idx)
    #     if fby_obj is None:
    #         raise ValueError(f"Brakuje FieldByYear #{fby_idx} dla semester_df[{idx}]")

    #     number = int(row["number"])
    #     ects_req = int(row.get("ects", 30) or 30)

    #     sem, created = Semester.objects.get_or_create(
    #         field_by_year=fby_obj,
    #         number=number,
    #         defaults={"ECTS_required": ects_req},
    #     )
    #     if not created and sem.ECTS_required != ects_req:
    #         sem.ECTS_required = ects_req
    #         sem.save(update_fields=["ECTS_required"])

    #     sem_map[idx] = sem
    #     print(f"  {'[+] nowy' if created else '[=] istnieje'} Semestr {idx}: {fby_obj.field.name} / {number}")

    # 4e. COURSE  ─────────────────────────────────────────────
    for idx, row in course_df.iterrows():
        fld_idx = int(row["field_id"])
        fld_obj = field_map.get(fld_idx)
        if fld_obj is None:
            raise ValueError(f"Brakuje kierunku #{fld_idx} dla course_df[{idx}]")

        crs, created = Course.objects.get_or_create(
            name=str(row["name"]).strip(),
            defaults={
                "ECTS":           int(row.get("ects", 0) or 0),
                "test_type":      row.get("test_type", "egzamin") or "egzamin",
                "additional_info": row.get("description", "") or "",
                "lecturer":        row.get("lecturer", "") or "",
            },
        )
        # aktualizuj miękkie pola, jeśli obiekt już był
        if not created:
            dirty = False
            for attr, col in (
                ("ECTS", "ects"),
                ("test_type", "test_type"),
                ("additional_info", "description"),
                ("lecturer", "lecturer"),
            ):
                new_val = row.get(col, "") or ""
                if attr == "ECTS" and new_val != "":
                    new_val = int(new_val)
                if getattr(crs, attr) != new_val:
                    setattr(crs, attr, new_val); dirty = True
            if dirty:
                crs.save()

        # Many-to-Many: Course ↔ Field
        if not crs.field.filter(id=fld_obj.id).exists():
            crs.field.add(fld_obj)

        # powiązanie Course ↔ Semester (jeśli mamy numer semestru)
        if pd.notna(row["semester"]):
            sem_no = int(row["semester"])
            # wyszukaj semestr w DB (może kilka roczników – weź wszystkie)
            sem_list = Semester.objects.filter(field_by_year__field=fld_obj, number=sem_no)
            for sem in sem_list:
                if not crs.semester.filter(id=sem.id).exists():
                    crs.semester.add(sem)

        print(f"  {'[+] nowy' if created else '[=] istnieje'} Kurs {idx}: {crs.name}")

print("\n✅ Import zakończony pomyślnie – brak duplikatów.")

Migrations for 'mainApp':
  mainApp\migrations\0016_alter_group_code.py
    - Alter field code on group
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, mainApp, sessions
Running migrations:
  Applying mainApp.0015_field_description_field_specialization_field_type_and_more...

IntegrityError: The row in table 'mainApp_field_G2_subject' with primary key '16' has an invalid foreign key: mainApp_field_G2_subject.field_id contains a value '185' that does not have a corresponding value in mainApp_field.id.